In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
%matplotlib inline

In [2]:
! git clone https://github.com/TStarkenburg/GalaxyMergerExample.git

Cloning into 'GalaxyMergerExample'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 50 (delta 1), reused 44 (delta 0), pack-reused 0
Unpacking objects: 100% (50/50), done.
Checking out files: 100% (42/42), done.


In [3]:
! mkdir images


In [ ]:
for snapshot in range(81):
    snapfile = h5.File('GalaxyMergerExample/data/stars_snapshot_%03d.hdf5' %(snapshot*5, snapshot*5), 'r')
    print ("opening snapshot", snapshot)

    pos1 = snapfile['PartType2/Coordinates'][:,:]
    pos2 = snapfile['PartType3/Coordinates'][:,:]
    if snapshot > 0:
        posn = snapfile['PartType4/Coordinates'][:,:]
        massn = snapfile['PartType4/Mass'][:]*1.0e10/0.73
        pos = np.concatenate((pos1, pos2, posn), axis = 0)
        mass = np.concatenate((np.repeat(2.0e-8*1.0e10/0.73, len(pos1[:,0])),np.repeat(2.0e-8*1.0e10/0.73, len(pos2[:,0])),massn),axis = 0)
    else:
        pos = np.concatenate((pos1, pos2), axis = 0)
        mass = np.concatenate((np.repeat(2.0e-8*1.0e10/0.73, len(pos1[:,0])),np.repeat(2.0e-8*1.0e10/0.73, len(pos2[:,0]))),axis = 0)

    plt.subplots(1,1,figsize=[8,6])
    maxsize = 60
    nbins = 60
    binopp = (2.0*1000*maxsize/nbins)**2 #in pc^2
    histdisk, xedges, yedges = np.histogram2d(-pos[:,1], pos[:,0], range = [[-20, 40],[-40, 20]], bins=nbins, weights = mass[:]*1.0e10/0.73/binopp)
    plt.imshow(np.log10(histdisk*(histdisk > 10*2.0e-8*1.0e10/0.73/binopp)), extent = (-40, 20, -40, 20), cmap='plasma', vmin = 9.5, vmax = 12.5)
    plt.text(-35, 15, 'time = %0.0f Million yr' %(snapshot*10*5), color = 'black')
    cbar1 = plt.colorbar()
    plt.axis([-40, 20, -40, 20])
    plt.xlabel('x [kpc]')
    plt.ylabel('y [kpc]')
    plt.title('Stars')
    cbar1.set_label('Log Surface Density [Msun/pc^2]', rotation=90)
    plt.savefig('images/stars_%03d.png' %snapshot)
    plt.close()

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

video_name = 'stars.mp4'

images = []
for i in range(151):
    images.append('images/stars_%03d.png' %(i*2))

# Determine the width and height from the first image
frame = cv2.imread(images[0])
#cv2_imshow(frame)
height, width, channels = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter(video_name, fourcc, 20.0, (width, height))

for image in images:
    frame = cv2.imread(image)

    out.write(frame) # Write out frame to video

    #cv2_imshow(frame)
    if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
        break

# Release everything if job is finished
out.release()
cv2.destroyAllWindows()

print("The output video is {}".format(video_name))

The output video is darkMatter_i2.mp4
